## JSON 형태 Parsing하기
### Reference 
* https://www.kaggle.com/yingzhou0510/parsing-json-format-data-for-beginners
* https://www.kaggle.com/chumajin/eda-of-mlb-for-starter-english-ver

### 1. 한 칼럼 당 JSON Parsing하기

In [12]:
import os.path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

In [4]:
file_path = ".../MLB_FAN_ENGAGEMENT/data"

example_test = pd.read_csv(file_path + '/example_test.csv')
example_test

,date,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20210426,"[{""gamePk"":634374,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-26"",""t...","[{""home"":1,""gamePk"":634377,""gameDate"":""2021-04...","[{""home"":1,""teamId"":139,""gamePk"":634343,""gameD...","[{""transactionId"":480386,""playerId"":543685,""pl...","[{""season"":2021,""gameDate"":""2021-04-26"",""divis...",NaN,"[{""gamePk"":634433,""gameDate"":""2021-04-26"",""gam...",NaN,NaN
1,20210427,"[{""gamePk"":634318,""gameType"":""R"",""season"":2021...","[{""playerId"":443558,""gameDate"":""2021-04-27"",""t...","[{""home"":1,""gamePk"":634320,""gameDate"":""2021-04...","[{""home"":1,""teamId"":117,""gamePk"":634333,""gameD...","[{""transactionId"":480456,""playerId"":642162,""pl...","[{""season"":2021,""gameDate"":""2021-04-27"",""divis...",NaN,"[{""gamePk"":634332,""gameDate"":""2021-04-27"",""gam...",NaN,NaN
2,20210428,"[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
3,20210429,"[{""gamePk"":634330,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-29"",""t...","[{""home"":1,""gamePk"":634330,""gameDate"":""2021-04...","[{""home"":0,""teamId"":119,""gamePk"":634346,""gameD...","[{""transactionId"":480993,""playerId"":606965,""pl...","[{""season"":2021,""gameDate"":""2021-04-29"",""divis...",NaN,"[{""gamePk"":634346,""gameDate"":""2021-04-29"",""gam...",NaN,NaN
4,20210430,"[{""gamePk"":634287,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-30"",""t...","[{""home"":1,""gamePk"":634305,""gameDate"":""2021-04...","[{""home"":1,""teamId"":135,""gamePk"":634303,""gameD...",NaN,"[{""season"":2021,""gameDate"":""2021-04-30"",""divis...","[{""awardId"":""NLRRELMON"",""awardName"":""NL Reliev...","[{""gamePk"":634327,""gameDate"":""2021-04-30"",""gam...",NaN,NaN


In [14]:
# 'games' 칼럼 parsing
column_to_parse = ['games']

# parsing 시작
for col in column_to_parse:
    print(f"Parse columns {col}")

    # 데이터프레임 지정
    target_df = pd.DataFrame()
    target_df['date'] = example_test['date']
    
    for i in range(example_test.shape[0]):
        # 특정 칼럼의 셀 지정
        cell = example_test.iloc[i][col]
        
        if cell is not None: # null 제거
            try:
                value = json.loads(cell)
            except TypeError:
                continue
                
            for k, v in enumerate(value):
                prefix_col = f'{col}_{k}'
                
                for key, values in v.items():
                    new_col = f'{prefix_col}_{key}'
                    
                    # 데이터 타입을 다루기 위한 try/except
                    try:
                        target_df.at[i, new_col] = values
                    except ValueError:
                        target_df[new_col] = target_df[new_col].astype(object)

Parse columns games


In [16]:
target_df

,date,games_0_gamePk,games_0_gameType,games_0_season,games_0_gameDate,games_0_gameTimeUTC,games_0_resumeDate,games_0_resumedFrom,games_0_codedGameState,games_0_detailedGameState,...,games_14_homeWinner,games_14_homeScore,games_14_awayId,games_14_awayName,games_14_awayAbbrev,games_14_awayWins,games_14_awayLosses,games_14_awayWinPct,games_14_awayWinner,games_14_awayScore
0,20210426,634374.0,R,2021.0,2021-04-26,2021-04-27T01:45:00Z,NaN,NaN,F,Final,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210427,634318.0,R,2021.0,2021-04-27,2021-04-27T23:45:00Z,NaN,NaN,F,Final,...,False,5.0,115.0,Colorado Rockies,COL,9.0,14.0,0.391,True,7.0
2,20210428,634309.0,R,2021.0,2021-04-28,2021-04-29T01:45:00Z,NaN,NaN,F,Final,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210429,634330.0,R,2021.0,2021-04-29,2021-04-29T21:10:00Z,NaN,NaN,F,Final,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210430,634287.0,R,2021.0,2021-04-30,2021-05-01T02:10:00Z,NaN,NaN,F,Final,...,False,3.0,138.0,St. Louis Cardinals,STL,14.0,12.0,0.538,True,7.0


In [17]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 481 entries, date to games_14_awayScore
dtypes: float64(255), int64(1), object(225)
memory usage: 18.9+ KB


### 2. 더 간단한 Parser

In [18]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [22]:
# 'games' 칼럼 parsing
unpack_json(example_test['games'].iloc[0])

,gamePk,gameType,season,gameDate,gameTimeUTC,resumeDate,resumedFrom,codedGameState,detailedGameState,isTie,...,homeWinner,homeScore,awayId,awayName,awayAbbrev,awayWins,awayLosses,awayWinPct,awayWinner,awayScore
0,634374,R,2021,2021-04-26,2021-04-27T01:45:00Z,NaN,NaN,F,Final,False,...,True,12,115,Colorado Rockies,COL,8,14,0.364,False,0
1,634377,R,2021,2021-04-26,2021-04-27T00:05:00Z,NaN,NaN,F,Final,False,...,False,4,108,Los Angeles Angels,LAA,11,10,0.524,True,9
2,634433,R,2021,2021-04-26,2021-04-26T23:05:00Z,NaN,NaN,F,Final,False,...,True,4,147,New York Yankees,NYY,9,13,0.409,False,2
3,634363,R,2021,2021-04-26,2021-04-27T02:10:00Z,NaN,NaN,F,Final,False,...,False,3,113,Cincinnati Reds,CIN,10,12,0.455,True,5
4,634402,R,2021,2021-04-26,2021-04-26T23:10:00Z,NaN,NaN,F,Final,False,...,True,8,112,Chicago Cubs,CHC,10,12,0.455,False,7
5,634461,R,2021,2021-04-26,2021-04-26T23:40:00Z,NaN,NaN,F,Final,False,...,False,0,146,Miami Marlins,MIA,10,12,0.455,True,8
6,634343,R,2021,2021-04-26,2021-04-26T23:10:00Z,NaN,NaN,F,Final,False,...,False,1,133,Oakland Athletics,OAK,15,8,0.652,True,2
7,634345,R,2021,2021-04-26,2021-04-26T23:45:00Z,NaN,NaN,F,Final,False,...,False,1,143,Philadelphia Phillies,PHI,11,11,0.500,True,2
8,634382,R,2021,2021-04-26,2021-04-27T00:10:00Z,NaN,NaN,F,Final,False,...,True,5,136,Seattle Mariners,SEA,13,10,0.565,False,2
9,634394,R,2021,2021-04-26,2021-04-26T22:10:00Z,NaN,NaN,F,Final,False,...,True,5,142,Minnesota Twins,MIN,7,14,0.333,False,3


In [25]:
# 'rosters' 칼럼 parsing
unpack_json(example_test['rosters'].iloc[0])

,playerId,gameDate,teamId,statusCode,status
0,405395,2021-04-26,108,A,Active
1,408234,2021-04-26,116,A,Active
2,444482,2021-04-26,109,A,Active
3,445276,2021-04-26,119,A,Active
4,446334,2021-04-26,137,A,Active
...,...,...,...,...,...
1264,676477,2021-04-26,111,A,Active
1265,676755,2021-04-26,110,RM,Reassigned to Minors
1266,676831,2021-04-26,138,RM,Reassigned to Minors
1267,676845,2021-04-26,136,RM,Reassigned to Minors


### 3. 정리
* 두 방식의 차이점이 존재한다. 
  - 1번 방식의 경우 그 날 치뤄진 경기를 날짜를 기준으로 각 경기 데이터를 칼럼으로 쭉 나열하는 방식 ('games'에 대한 하나의 완성된 데이터프레임이 됨)
  - 2번 방식은 각 날짜마다 치뤄진 경기를 뽑아낼 수 있음 ('games'의 4월 26일 경기들에 대한 데이터만 추출되므로 완성된 데이터프레임을 만들려면 각 날짜의 데이터를 추출하여 merge하는 방법을 거쳐야할 듯)
  - 어떤 방식이 더 좋은지는 모르겠지만, 데이터프레임을 자체를 보기에는 아무래도 2번의 경우가 나음.
  - 2번의 방식을 함수로 정리하면 범용성을 가진 parser로 사용할 수 있을 것으로 보임.